In [ ]:
try:
    cursor_sys_trend = coll_daily_sys.find({
        "metadata.timeWindow": "hourly",
        "timestamp": {"$gte": trend_start_dt, "$lt": yesterday_end_dt} # Fetch N days
    }).sort("timestamp", 1) # Sort by time for plotting

    records_sys_trend = list(cursor_sys_trend)

    if not records_sys_trend:
        print(f"No daily system records found for the last {N_DAYS_FOR_TRENDS} days. Skipping trend plots.")
    else:
        flat_records_sys_trend = flatten_metric_data(records_sys_trend)
        df_sys_trend = pd.DataFrame(flat_records_sys_trend)
        df_sys_trend['timestamp'] = pd.to_datetime(df_sys_trend['timestamp'])
        df_sys_trend = df_sys_trend.set_index('timestamp')
        cols=list(df_sys_trend.columns)

        print(f"Loaded {len(df_sys_trend)} daily system records for trend analysis.")

        # --- Plotting ---
        plt.style.use('seaborn-v0_8-whitegrid') # Use a nice style
        fig, axes = plt.subplots(len(cols), 1, figsize=(12, 5*len(cols)), sharex=True) # 4 plots vertically

        for i,col in enumerate(cols):
            
            for comp_id in df_sys_trend['metadata_companyID'].unique():
                df_comp = df_sys_trend[df_sys_trend['metadata_companyID'] == comp_id]
                axes[i].plot(df_comp.index, df_comp[col], marker='.', linestyle='-', label=f'Comp {comp_id}')
            axes[i].set_ylabel(col)
            axes[i].set_title(f'Hourly System {col}')
            axes[i].legend()
            axes[i].grid(True)


        # Improve date formatting on x-axis
        axes[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.xticks(rotation=45)
        plt.xlabel('Date')
        plt.tight_layout()
        plt.show()

except Exception as e: